In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Need for this homework
import geopandas as gpd
import contextily as cx

from shapely.geometry import Point

plt.style.use('default')
# This next line tells jupyter to render the images inline
%matplotlib inline
import matplotlib_inline
# This renders your figures as vector graphics AND gives you an option to download a PDF too
matplotlib_inline.backend_inline.set_matplotlib_formats('svg', 'pdf')


In [1]:
# Read in data
avy_df = pd.read_csv('./avalanches.csv')
#display(avy_df.head())
#display(avy_df['Coordinates'].head())

# Split the coordinates into two columns
avy_df[['Latitude', 'Longitude']] = avy_df['Coordinates'].str.split(',', expand=True)

# Convert the columns to numeric
avy_df['Latitude'] = pd.to_numeric(avy_df['Latitude'], errors='coerce')
avy_df['Longitude'] = pd.to_numeric(avy_df['Longitude'], errors='coerce')

# filter lat/long outliers
avy_filtered = avy_df[
    (avy_df['Latitude'] >= 36) & 
    (avy_df['Latitude'] <= 42) & 
    (avy_df['Longitude'] >= -114) & 
    (avy_df['Longitude'] <= -108)
]

NameError: name 'pd' is not defined

In [5]:
# Create a GeoDataFrame by converting 'Latitude' and 'Longitude' to geometry Points
geometry = [Point(xy) for xy in zip(avy_filtered['Longitude'], avy_filtered['Latitude'])]
imap = gpd.GeoDataFrame(avy_filtered, geometry=geometry)

# Set the coordinate reference system to WGS 84 (EPSG:4326)
imap.set_crs(epsg=4326, inplace=True)

# Reproject to Web Mercator (EPSG:3857) to match the basemap projection
imap = imap.to_crs(epsg=3857)

NameError: name 'Point' is not defined

In [ ]:
import folium